In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick
# import scipy.optimize as opt
# import scipy.stats as stat
# from mpl_toolkits.mplot3d import Axes3D
# from operator import itemgetter
# import random
# import numpy as np
# import numpy.ma as ma
# import numpy.linalg as la
# from mpl_toolkits.mplot3d import Axes3D

# pi = np.pi
# sin = np.sin
# cos = np.cos

In [2]:
import numpy as np
from MSD_utils import get_data

In [3]:
path = "./{concentration}_ACSF/geoM2xy_{sample_name}.csv"
data = {}

data = get_data(["RED", "YG"], ["PEG", "nPEG"], ["in_agarose"], ["0_1x", "1x", "10x"], ["S1", "S2", "S3", "S4"], path);

data

{'RED_PEG_in_agarose_0_1x_S1': array([         nan,   0.72494472,   1.24297782,   1.61163293,
          2.04402547,   2.40856821,   2.84479487,   3.29863632,
          3.68947942,   4.00950549,   4.28135613,   4.73680706,
          5.03638757,   5.37172446,   5.79354317,   6.03970001,
          6.25950618,   6.64833798,   6.58098093,   6.96809497,
          7.00301764,   7.60594357,   7.77516794,   8.19026243,
          8.07734583,   7.73915749,   8.10864936,   7.60760258,
          7.48229377,   8.39521054,   8.48061253,   8.2594792 ,
          7.68174466,   7.09616573,   7.72358358,   7.45837585,
          8.12863289,   8.09981136,   8.46195553,   6.71765095,
          4.48095503,   3.9362694 ,   4.45054164,   3.8946416 ,
          4.29304252,   4.45650338,   4.53917868,   3.89048816,
          3.95086542,   3.89157116,   5.38726858,   5.68768543,
          5.70628525,   5.64612967,   6.47431454,   7.9439325 ,
          8.11246849,   8.37451993,   7.54068867,   7.57251066,
          

In [15]:
def build_time_array(frames = 90, conversion = (0.16, 9.89, 1), SD_frames = [1, 7, 14]):
    """
    Builds (1) a time array based on the desire number of frames and the fps and (2) a shortened time array at which the
    standard deviations are going to be calculated.
    
    The default is 100 frames, a conversion factor of 0.16 microns per pixel, 9.89 frames per second, and 1 micron per z-stack.
    """
    
    frames_1 = np.linspace(0, frames, frames+1).astype(np.int64)
    time = frames_1/conversion[1]
    time_SD = np.zeros(np.size(SD_frames))

    for i in range(0, np.size(SD_frames)):
        time_SD[i] = time[SD_frames[i]]
    
    return time, time_SD

In [17]:
time, time_SD = build_time_array()

array([ 0.        ,  0.10111223,  0.20222447,  0.3033367 ,  0.40444894,
        0.50556117,  0.60667341,  0.70778564,  0.80889788,  0.91001011,
        1.01112235,  1.11223458,  1.21334681,  1.31445905,  1.41557128,
        1.51668352,  1.61779575,  1.71890799,  1.82002022,  1.92113246,
        2.02224469,  2.12335693,  2.22446916,  2.3255814 ,  2.42669363,
        2.52780586,  2.6289181 ,  2.73003033,  2.83114257,  2.9322548 ,
        3.03336704,  3.13447927,  3.23559151,  3.33670374,  3.43781598,
        3.53892821,  3.64004044,  3.74115268,  3.84226491,  3.94337715,
        4.04448938,  4.14560162,  4.24671385,  4.34782609,  4.44893832,
        4.55005056,  4.65116279,  4.75227503,  4.85338726,  4.95449949,
        5.05561173,  5.15672396,  5.2578362 ,  5.35894843,  5.46006067,
        5.5611729 ,  5.66228514,  5.76339737,  5.86450961,  5.96562184,
        6.06673407,  6.16784631,  6.26895854,  6.37007078,  6.47118301,
        6.57229525,  6.67340748,  6.77451972,  6.87563195,  6.97

In [ ]:
def return_average(n1, n2, n3, n4):

    f1 = np.zeros((frames, 4))

    for i in range(0, frames):
        f1[i, 0] = n1[i]
        f1[i, 1] = n2[i]
        f1[i, 2] = n3[i]
        f1[i, 3] = n4[i]
        
    answer = np.mean(f1, axis=1)
    return answer

In [ ]:
geo_x01_P_Y = return_average(geo_x01_P_Y_S1, geo_x01_P_Y_S2, geo_x01_P_Y_S3, geo_x01_P_Y_S4)
geo_x01_P_R = return_average(geo_x01_P_R_S1, geo_x01_P_R_S2, geo_x01_P_R_S3, geo_x01_P_R_S4)
geo_x01_N_Y = return_average(geo_x01_N_Y_S1, geo_x01_N_Y_S2, geo_x01_N_Y_S3, geo_x01_N_Y_S4)
geo_x01_N_R = return_average(geo_x01_N_R_S1, geo_x01_N_R_S2, geo_x01_N_R_S3, geo_x01_N_R_S4)

geo_x1_P_Y = return_average(geo_x1_P_Y_S1, geo_x1_P_Y_S2, geo_x1_P_Y_S3, geo_x1_P_Y_S4)
geo_x1_P_R = return_average(geo_x1_P_R_S1, geo_x1_P_R_S2, geo_x1_P_R_S3, geo_x1_P_R_S4)
geo_x1_N_Y = return_average(geo_x1_N_Y_S1, geo_x1_N_Y_S2, geo_x1_N_Y_S3, geo_x1_N_Y_S4)
geo_x1_N_R = return_average(geo_x1_N_R_S1, geo_x1_N_R_S2, geo_x1_N_R_S3, geo_x1_N_R_S4)

geo_x10_P_Y = return_average(geo_x10_P_Y_S1, geo_x10_P_Y_S2, geo_x10_P_Y_S3, geo_x10_P_Y_S4)
geo_x10_P_R = return_average(geo_x10_P_R_S1, geo_x10_P_R_S2, geo_x10_P_R_S3, geo_x10_P_R_S4)
geo_x10_N_Y = return_average(geo_x10_N_Y_S1, geo_x10_N_Y_S2, geo_x10_N_Y_S3, geo_x10_N_Y_S4)
geo_x10_N_R = return_average(geo_x10_N_R_S1, geo_x10_N_R_S2, geo_x10_N_R_S3, geo_x10_N_R_S4)

In [ ]:
geo_err1 = np.zeros(3)
geo_err2 = np.zeros(3)
geo_err3 = np.zeros(3)
geo_err4 = np.zeros(3)
geo_err5 = np.zeros(3)
geo_err6 = np.zeros(3)
geo_err7 = np.zeros(3)
geo_err8 = np.zeros(3)
geo_err9 = np.zeros(3)
geo_err10 = np.zeros(3)
geo_err11 = np.zeros(3)
geo_err12 = np.zeros(3)

for i in range(0,3):
    geo_err1[i] = geo_x01_P_Y[SD_frames[i]]
    geo_err2[i] = geo_x01_P_R[SD_frames[i]]
    geo_err3[i] = geo_x01_N_Y[SD_frames[i]]
    geo_err4[i] = geo_x01_N_R[SD_frames[i]]

    geo_err5[i] = geo_x1_P_Y[SD_frames[i]]
    geo_err6[i] = geo_x1_P_R[SD_frames[i]]
    geo_err7[i] = geo_x1_N_Y[SD_frames[i]]
    geo_err8[i] = geo_x1_N_R[SD_frames[i]]

    geo_err9[i] = geo_x10_P_Y[SD_frames[i]]
    geo_err10[i] = geo_x10_P_R[SD_frames[i]]
    geo_err11[i] = geo_x10_N_Y[SD_frames[i]]
    geo_err12[i] = geo_x10_N_R[SD_frames[i]]

In [ ]:
def return_SD(n1, n2, n3, n4):
    f1 = np.zeros((frames, 4))
    a = 4.*0.5

    for i in range(0, frames):
        f1[i, 0] = n1[i]
        f1[i, 1] = n2[i]
        f1[i, 2] = n3[i]
        f1[i, 3] = n4[i]

    answer1 = np.std(f1, axis=1)
    answer = np.zeros(3)
    for i in range(0, 3):
        answer[i] = answer1[SD_frames[i]]/a
    return answer

In [ ]:
std_x01_P_Y = return_SD(geo_x01_P_Y_S1, geo_x01_P_Y_S2, geo_x01_P_Y_S3, geo_x01_P_Y_S4)
std_x01_P_R = return_SD(geo_x01_P_R_S1, geo_x01_P_R_S2, geo_x01_P_R_S3, geo_x01_P_R_S4)
std_x01_N_Y = return_SD(geo_x01_N_Y_S1, geo_x01_N_Y_S2, geo_x01_N_Y_S3, geo_x01_N_Y_S4)
std_x01_N_R = return_SD(geo_x01_N_R_S1, geo_x01_N_R_S2, geo_x01_N_R_S3, geo_x01_N_R_S4)

std_x1_P_Y = return_SD(geo_x1_P_Y_S1, geo_x1_P_Y_S2, geo_x1_P_Y_S3, geo_x1_P_Y_S4)
std_x1_P_R = return_SD(geo_x1_P_R_S1, geo_x1_P_R_S2, geo_x1_P_R_S3, geo_x1_P_R_S4)
std_x1_N_Y = return_SD(geo_x1_N_Y_S1, geo_x1_N_Y_S2, geo_x1_N_Y_S3, geo_x1_N_Y_S4)
std_x1_N_R = return_SD(geo_x1_N_R_S1, geo_x1_N_R_S2, geo_x1_N_R_S3, geo_x1_N_R_S4)

std_x10_P_Y = return_SD(geo_x10_P_Y_S1, geo_x10_P_Y_S2, geo_x10_P_Y_S3, geo_x10_P_Y_S4)
std_x10_P_R = return_SD(geo_x10_P_R_S1, geo_x10_P_R_S2, geo_x10_P_R_S3, geo_x10_P_R_S4)
std_x10_N_Y = return_SD(geo_x10_N_Y_S1, geo_x10_N_Y_S2, geo_x10_N_Y_S3, geo_x10_N_Y_S4)
std_x10_N_R = return_SD(geo_x10_N_R_S1, geo_x10_N_R_S2, geo_x10_N_R_S3, geo_x10_N_R_S4)

In [ ]:
#Figure parameters

filename = '01x.png'

frame_m = 15

limit1=1.51 #x range
tick1 = 0.5

limit2=40.1 #y range
tick2 = 8 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=1 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geo_x01_N_R[0:frame_m], 'g-', linewidth=wi, label='100nm')
ax.plot(time[0:frame_m], geo_x01_P_R[0:frame_m], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm PEG')
ax.plot(time[0:frame_m], geo_x01_N_Y[0:frame_m], 'b-', linewidth=wi, label='60nm')
ax.plot(time[0:frame_m], geo_x01_P_Y[0:frame_m], 'b--', linewidth=wi, dashes=[d1,d2], label='60nm PEG')

ax.errorbar(time_SD, geo_err4, yerr=std_x01_N_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err2, yerr=std_x01_P_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err3, yerr=std_x01_N_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
ax.errorbar(time_SD, geo_err1, yerr=std_x01_P_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')


# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = '1x.png'

frame_m = 15

limit1=1.51 #x range
tick1 = 0.5

limit2=40.1 #y range
tick2 = 8 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=1 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geo_x1_N_R[0:frame_m], 'g-', linewidth=wi, label='100nm')
ax.plot(time[0:frame_m], geo_x1_P_R[0:frame_m], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm PEG')
ax.plot(time[0:frame_m], geo_x1_N_Y[0:frame_m], 'b-', linewidth=wi, label='60nm')
ax.plot(time[0:frame_m], geo_x1_P_Y[0:frame_m], 'b--', linewidth=wi, dashes=[d1,d2], label='60nm PEG')

ax.errorbar(time_SD, geo_err8, yerr=std_x01_N_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err6, yerr=std_x01_P_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err7, yerr=std_x01_N_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
ax.errorbar(time_SD, geo_err5, yerr=std_x01_P_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')


# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = '10x.png'

frame_m = 15

limit1=1.51 #x range
tick1 = 0.5

limit2=40.1 #y range
tick2 = 8 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=1 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geo_x10_N_R[0:frame_m], 'g-', linewidth=wi, label='100nm')
ax.plot(time[0:frame_m], geo_x10_P_R[0:frame_m], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm PEG')
ax.plot(time[0:frame_m], geo_x10_N_Y[0:frame_m], 'b-', linewidth=wi, label='60nm')
ax.plot(time[0:frame_m], geo_x10_P_Y[0:frame_m], 'b--', linewidth=wi, dashes=[d1,d2], label='60nm PEG')

ax.errorbar(time_SD, geo_err12, yerr=std_x01_N_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err10, yerr=std_x01_P_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err11, yerr=std_x01_N_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
ax.errorbar(time_SD, geo_err9, yerr=std_x01_P_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')


# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'PEG_RED.png'

frame_m = 15

limit1=1.51 #x range
tick1 = 0.5

limit2=10.1 #y range
tick2 = 2 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=1 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geo_x01_P_R[0:frame_m], 'g-', linewidth=wi, label='0.1x')
ax.plot(time[0:frame_m], geo_x1_P_R[0:frame_m], 'b-', linewidth=wi, label='1x')
ax.plot(time[0:frame_m], geo_x10_P_R[0:frame_m], 'r-', linewidth=wi, label='10x')

ax.errorbar(time_SD, geo_err2, yerr=std_x01_P_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err6, yerr=std_x1_P_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
ax.errorbar(time_SD, geo_err10, yerr=std_x10_P_R, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')


# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'PEG_YG.png'

frame_m = 15

limit1=1.51 #x range
tick1 = 0.5

limit2=40.1 #y range
tick2 = 8 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=1 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geo_x01_P_Y[0:frame_m], 'g-', linewidth=wi, label='0.1x')
ax.plot(time[0:frame_m], geo_x1_P_Y[0:frame_m], 'b-', linewidth=wi, label='1x')
ax.plot(time[0:frame_m], geo_x10_P_Y[0:frame_m], 'r-', linewidth=wi, label='10x')

ax.errorbar(time_SD, geo_err1, yerr=std_x01_P_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err5, yerr=std_x1_P_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
ax.errorbar(time_SD, geo_err9, yerr=std_x10_P_Y, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')


# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
std_x01_N_R

In [ ]:
time_SD

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geoRED_WT_P1_mid[0:frame_m], 'g--', linewidth=wi, dashes=[d1,d2], label='P1')
ax.plot(time[0:frame_m], geoRED_WT_P2_mid[0:frame_m], 'b--', linewidth=wi, dashes=[d1,d2], label='P2')
ax.plot(time[0:frame_m], geoRED_WT_P3_mid[0:frame_m], 'r--', linewidth=wi, dashes=[d1,d2], label='P3')

# ax.errorbar(time_SD, geo_err1, yerr=stdRED_cor, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
# ax.errorbar(time_SD, geo_err2, yerr=stdRED_hip, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(time_SD, geo_err3, yerr=stdRED_mid, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.82), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'YG_pH.png'

frame_m = 11

limit1=2.1 #x range
tick1 = 0.5

limit2=120.1 #y range
tick2 = 30 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=1 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[0:frame_m], geoYG_pH6[0:frame_m], 'g--', linewidth=wi, dashes=[d1,d2], label='pH6')
ax.plot(time[0:frame_m], geoYG_pH7[0:frame_m], 'b--', linewidth=wi, dashes=[d1,d2], label='pH7')
ax.plot(time[0:frame_m], geoYG_pH8[0:frame_m], 'r--', linewidth=wi, dashes=[d1,d2], label='pH8')

ax.errorbar(time_SD, geo_err1a, yerr=stdYG_pH6, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
ax.errorbar(time_SD, geo_err2a, yerr=stdYG_pH7, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
ax.errorbar(time_SD, geo_err3a, yerr=stdYG_pH8, fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.80), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Repeat for x data

xgeoRED_pH6_1 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH6_1), delimiter =",")
xgeoRED_pH6_2 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH6_2), delimiter =",")
xgeoRED_pH6_3 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH6_3), delimiter =",")

xgeoRED_pH7_1 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH7_1), delimiter =",")
xgeoRED_pH7_2 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH7_2), delimiter =",")
xgeoRED_pH7_3 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH7_3), delimiter =",")

xgeoRED_pH8_1 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH8_1), delimiter =",")
xgeoRED_pH8_2 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH8_2), delimiter =",")
xgeoRED_pH8_3 = np.genfromtxt('geoM1x_{}.csv'.format(RED_pH8_3), delimiter =",")

xgeoYG_pH6_1 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH6_1), delimiter =",")
xgeoYG_pH6_2 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH6_2), delimiter =",")
xgeoYG_pH6_3 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH6_3), delimiter =",")

xgeoYG_pH7_1 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH7_1), delimiter =",")
xgeoYG_pH7_2 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH7_2), delimiter =",")
xgeoYG_pH7_3 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH7_3), delimiter =",")

xgeoYG_pH8_1 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH8_1), delimiter =",")
xgeoYG_pH8_2 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH8_2), delimiter =",")
xgeoYG_pH8_3 = np.genfromtxt('geoM1x_{}.csv'.format(YG_pH8_3), delimiter =",")

In [ ]:
#Repeat for x data

ygeoRED_pH6_1 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH6_1), delimiter =",")
ygeoRED_pH6_2 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH6_2), delimiter =",")
ygeoRED_pH6_3 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH6_3), delimiter =",")

ygeoRED_pH7_1 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH7_1), delimiter =",")
ygeoRED_pH7_2 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH7_2), delimiter =",")
ygeoRED_pH7_3 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH7_3), delimiter =",")

ygeoRED_pH8_1 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH8_1), delimiter =",")
ygeoRED_pH8_2 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH8_2), delimiter =",")
ygeoRED_pH8_3 = np.genfromtxt('geoM1y_{}.csv'.format(RED_pH8_3), delimiter =",")

ygeoYG_pH6_1 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH6_1), delimiter =",")
ygeoYG_pH6_2 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH6_2), delimiter =",")
ygeoYG_pH6_3 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH6_3), delimiter =",")

ygeoYG_pH7_1 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH7_1), delimiter =",")
ygeoYG_pH7_2 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH7_2), delimiter =",")
ygeoYG_pH7_3 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH7_3), delimiter =",")

ygeoYG_pH8_1 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH8_1), delimiter =",")
ygeoYG_pH8_2 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH8_2), delimiter =",")
ygeoYG_pH8_3 = np.genfromtxt('geoM1y_{}.csv'.format(YG_pH8_3), delimiter =",")

In [ ]:
#Linear algebra to find Deff:
geoM2xy = geoYG_pH8_3
geoM1x = xgeoYG_pH8_3
geoM1y = ygeoYG_pH8_3

w = dict()
residuals = dict()
line = dict()
A1 = np.ones((np.shape(time[1:frame_m])[0], 2))
A1[:, 0] = time[1:frame_m]

w[0], residuals[0] = np.linalg.lstsq(A1[:, :], geoM2xy[1:frame_m])[0:2]
w[1], residuals[1] = np.linalg.lstsq(A1[:, :], geoM1x[1:frame_m])[0:2]
w[2], residuals[2] = np.linalg.lstsq(A1[:, :], geoM1y[1:frame_m])[0:2]


line[0] = w[0][0]*time + w[0][1]

In [ ]:
print('2D Diffusion Coeff (um^2/s):', np.round(w[0][0]/4, 5), 'geoM2xy')
print('1Dx Diffusion Coeff (um^2/s):', np.round(w[1][0]/2, 5), 'geoM1x')
print('1Dy Diffusion Coeff (um^2/s):', np.round(w[2][0]/2, 5), 'geoM1y')